# Removing Filters from Convolutional Layers

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics
import time

In [3]:
cifar = tf.keras.datasets.cifar10
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
y_train, y_test = tf.one_hot(y_train.flatten(), 10), tf.one_hot(y_test.flatten(), 10)
y_test_flat = np.argmax(y_test, axis=1)

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [19]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
    
tester_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [21]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.7206 - accuracy: 0.3680 - auc_4: 0.8209
Epoch 2/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.3715 - accuracy: 0.4994 - auc_4: 0.8929
Epoch 3/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.2415 - accuracy: 0.5440 - auc_4: 0.9128
Epoch 4/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1366 - accuracy: 0.5842 - auc_4: 0.9271
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.0497 - accuracy: 0.6189 - auc_4: 0.9378
Epoch 6/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9677 - accuracy: 0.6463 - auc_4: 0.9471
Epoch 7/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8923 - accuracy: 0.6777 - auc_4: 0.9548
Epoch 8/10
1563/1563 [==============================] - ETA: 0s - loss: 0.8185 - accuracy: 0.7022 - auc_4: 0.96 - 9s 6ms/step - loss: 0.8186 - accuracy: 0.7021 - auc_4: 0.961

In [22]:
l, a, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model.get_weights()
weight_len = len(or_weights) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
dense_layer_sizes = [64]
conv_layer_sizes = [128]
conv_len = weight_len - 2 * len(dense_layer_sizes)

#For-loop over convolutional layers
for layer, size in enumerate(conv_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[conv_len - (2*layer+1)][:, :, :, i] = 0
        w[conv_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

#For-loop over dense layers
for layer, size in enumerate(dense_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

40/40 - 0s - loss: 1.3278 - accuracy: 0.5847 - auc_4: 0.9126
Node 0: -0.0014998912811279297
Node 1: 9.822845458984375e-05
Node 2: 0.0001723766326904297
Node 3: 0.0
Node 4: -0.0011245012283325195
Node 5: 0.0
Node 6: 0.0005205869674682617
Node 7: 0.003957152366638184
Node 8: 0.004172801971435547
Node 9: 0.0
Node 10: 0.0009812116622924805
Node 11: 0.0
Node 12: 0.007231950759887695
Node 13: 0.0
Node 14: 0.0
Node 15: -0.048014163970947266
Node 16: 0.0
Node 17: 2.7298927307128906e-05
Node 18: -0.02084052562713623
Node 19: 0.0
Node 20: -0.026913046836853027
Node 21: 0.00026988983154296875
Node 22: -0.0019714832305908203
Node 23: 0.0
Node 24: 0.008685946464538574
Node 25: -0.0003619194030761719
Node 26: 6.556510925292969e-06
Node 27: 0.016967415809631348
Node 28: 1.1920928955078125e-07
Node 29: -0.009758353233337402
Node 30: 0.0011377334594726562
Node 31: 0.0
Node 32: 0.00048220157623291016
Node 33: -1.5497207641601562e-06
Node 34: 0.001094222068786621
Node 35: 0.0026895999908447266
Node 36: -

In [23]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 57
Worse Nodes: 1
Important Nodes: 6
######### LAYER 1 #########
Zero Nodes: 38
Worse Nodes: 45
Important Nodes: 45


In [31]:
loss, acc, auc = model.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model.get_weights()
tol = -1e-5
layer_sizes = [64]
conv_layer_sizes = [128]
bas2 = [acc]
bls2 = [loss]
best_weights4 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts3 = []
places3 = []
for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights4)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights4[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights4[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts3.append(num_removed2)
    places3.append(nodes_removed2)

for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights4)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model.set_weights(w)
        nl, na, nauc = tester_model.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights4[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights4[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts3.append(num_removed2)
    places3.append(nodes_removed2)

tester_model.set_weights(best_weights4)

20/20 - 0s - loss: 1.3278 - accuracy: 0.5847 - auc_4: 0.9126
Considering layer 1
Improvement has occured!! Accuracy: 0.5846999883651733 --- Loss: 1.3277618885040283 --- Change: -8.344650268554687e-08 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5846999883651733 --- Loss: 1.3277618885040283 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5846999883651733 --- Loss: 1.3272415399551392 --- Change: 0.0003642439842224121 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5827000141143799 --- Loss: 1.3232766389846802 --- Change: 0.002175438404083252 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5828999876976013 --- Loss: 1.3231372833251953 --- Change: 0.00015754103660583496 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5825999975204468 --- Loss: 1.3188047409057617 --- Change: 0.002942782640457153 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.5825999975204468 --- Loss: 1.3188047409057617 --- Change: 0.0 --- New t

In [37]:
tester_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 1.2484 - accuracy: 0.5855 - auc_5: 0.9178


[1.2483556270599365, 0.5855000019073486, 0.9177575707435608]

In [33]:
y_pred = model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.638, 0.642, 0.434, 0.404, 0.582, 0.413, 0.676, 0.621, 0.741, 0.696]


In [34]:
y_pred = tester_model.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.643, 0.746, 0.381, 0.357, 0.59, 0.45, 0.709, 0.644, 0.689, 0.646]


In [35]:
amounts3

[76, 58]

In [36]:
layer_sizes

[6]

In [38]:
conv_layer_sizes

[52]

In [11]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [12]:
tester_model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [13]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4804 - accuracy: 0.4579 - auc_4: 0.8714 - val_loss: 1.1099 - val_accuracy: 0.6023 - val_auc_4: 0.9299
Epoch 2/5
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9963 - accuracy: 0.6485 - auc_4: 0.9435 - val_loss: 0.9322 - val_accuracy: 0.6728 - val_auc_4: 0.9510
Epoch 3/5
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7969 - accuracy: 0.7199 - auc_4: 0.9632 - val_loss: 0.9150 - val_accuracy: 0.6867 - val_auc_4: 0.9516
Epoch 4/5
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6647 - accuracy: 0.7676 - auc_4: 0.9740 - val_loss: 0.8108 - val_accuracy: 0.7237 - val_auc_4: 0.9614
Epoch 5/5
1563/1563 [==============================] - 6s 4ms/step - loss: 0.5525 - accuracy: 0.8058 - auc_4: 0.9815 - val_loss: 0.7984 - val_accuracy: 0.7392 - val_auc_4: 0.9628


In [14]:
model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.7984 - accuracy: 0.7392 - auc_4: 0.9628


[0.7984009981155396, 0.7391999959945679, 0.9627801775932312]

In [15]:
l, a, auc = model2.evaluate(x_train, y_train, verbose=2, batch_size=256)
or_weights2 = model2.get_weights()
weight_len = len(or_weights2) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
dense_layer_sizes = [64]
conv_layer_sizes = [256]
conv_len = weight_len - 2 * len(dense_layer_sizes)

#For-loop over dense layers
for layer, size in enumerate(dense_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights2)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_train, y_train, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))
    
#For-loop over convolutional layers
for layer, size in enumerate(conv_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights2)
        w[conv_len - (2*layer+1)][:, :, :, i] = 0
        w[conv_len - 2*layer][i] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_train, y_train, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

196/196 - 1s - loss: 0.4232 - accuracy: 0.8534 - auc_4: 0.9893
Node 0: 0.0
Node 1: 0.0
Node 2: 0.0013521015644073486
Node 3: -3.5762786865234375e-07
Node 4: 0.0
Node 5: 0.0
Node 6: 0.0
Node 7: -0.07783263921737671
Node 8: -1.8596649169921875e-05
Node 9: 0.0
Node 10: -1.4901161193847656e-07
Node 11: -0.018270939588546753
Node 12: 0.0
Node 13: -0.0499434769153595
Node 14: -0.14751791954040527
Node 15: -0.2636755704879761
Node 16: 0.0
Node 17: 0.0
Node 18: 0.0
Node 19: 0.0
Node 20: -0.18876487016677856
Node 21: 0.0
Node 22: 0.0
Node 23: -7.331371307373047e-06
Node 24: 0.0
Node 25: -9.08970832824707e-06
Node 26: 0.0
Node 27: -0.06157222390174866
Node 28: 0.0
Node 29: -8.383393287658691e-05
Node 30: 0.0
Node 31: 0.0
Node 32: 0.0
Node 33: 0.0
Node 34: -0.2191484570503235
Node 35: 0.0
Node 36: -0.0007567107677459717
Node 37: 0.0
Node 38: 0.0
Node 39: 0.0
Node 40: 0.0
Node 41: 0.0
Node 42: 0.0
Node 43: -0.10855263471603394
Node 44: -0.3271961808204651
Node 45: -0.27273261547088623
Node 46: 0.0

In [16]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 84
Worse Nodes: 22
Important Nodes: 150
######### LAYER 1 #########
Zero Nodes: 43
Worse Nodes: 1
Important Nodes: 20


In [17]:
l, a, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights2 = model2.get_weights()
weight_len = len(or_weights2) - 3
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
amounts = []
places = []
dense_layer_sizes = [64]
conv_layer_sizes = [256]
conv_len = weight_len - 2 * len(dense_layer_sizes)

#For-loop over dense layers
for layer, size in enumerate(dense_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights2)
        w[weight_len - (2*layer+1)][:,i] = 0
        w[weight_len - 2*layer][i] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))
    
#For-loop over convolutional layers
for layer, size in enumerate(conv_layer_sizes):
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    for i in range(size):
        w = copy.deepcopy(or_weights2)
        w[conv_len - (2*layer+1)][:, :, :, i] = 0
        w[conv_len - 2*layer][i] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
        print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
        change = l - nl
        if change <= tol_high and change >= tol_low:
            num_zeros += 1
            z += [i]
        elif change > 0:
            num_worse += 1
            wr += [i]
        else:
            num_important += 1
            imp += [i]
    amounts.append((num_zeros, num_worse, num_important))
    places.append((z, wr, imp))

40/40 - 0s - loss: 0.7984 - accuracy: 0.7392 - auc_4: 0.9628
Node 0: 0.0
Node 1: 0.0
Node 2: 0.008527517318725586
Node 3: 0.0
Node 4: 0.0
Node 5: 0.0
Node 6: 0.0
Node 7: -0.061670780181884766
Node 8: 0.0
Node 9: 0.0
Node 10: 0.0
Node 11: -0.009320676326751709
Node 12: 0.0
Node 13: -0.029446899890899658
Node 14: -0.11987334489822388
Node 15: -0.2563565969467163
Node 16: 0.0
Node 17: 0.0
Node 18: 0.0
Node 19: 0.0
Node 20: -0.1815524697303772
Node 21: 0.0
Node 22: 0.0
Node 23: -5.960464477539063e-08
Node 24: 0.0
Node 25: -2.9206275939941406e-06
Node 26: 0.0
Node 27: -0.052210450172424316
Node 28: 0.0
Node 29: -0.00041747093200683594
Node 30: 0.0
Node 31: 0.0
Node 32: 0.0
Node 33: 0.0
Node 34: -0.2058887481689453
Node 35: 0.0
Node 36: -0.00036913156509399414
Node 37: 0.0
Node 38: 0.0
Node 39: 0.0
Node 40: 0.0
Node 41: 0.0
Node 42: 0.0
Node 43: -0.09705352783203125
Node 44: -0.31480371952056885
Node 45: -0.27714407444000244
Node 46: 0.0
Node 47: 0.0
Node 48: 0.0
Node 49: 0.0
Node 50: -0.266

In [18]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 85
Worse Nodes: 90
Important Nodes: 81
######### LAYER 1 #########
Zero Nodes: 45
Worse Nodes: 2
Important Nodes: 17


In [10]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
tol = -1e-5
layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []

for layer, size in enumerate(layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.3*(na - oa) + 0.7*(ol - nl) >= tol:
            best_change = 0.3*(na - oa) + 0.7*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)

tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7811 - accuracy: 0.7343 - auc: 0.9641
Considering layer 1
Improvement has occured!! Accuracy: 0.7343000173568726 --- Loss: 0.7811414003372192 --- Change: 4.1723251342773435e-08 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7336999773979187 --- Loss: 0.7791403532028198 --- Change: 0.0012207210063934326 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7336999773979187 --- Loss: 0.7791403532028198 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7333999872207642 --- Loss: 0.7788729667663574 --- Change: 9.717345237731933e-05 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7333999872207642 --- Loss: 0.7788729667663574 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7343999743461609 --- Loss: 0.7716400623321533 --- Change: 0.0053630292415618895 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7343000173568726 --- Loss: 0.7700976133346558 --- Change: 0.001049727201461792 --- New tol

NameError: name 'best_weights4' is not defined

In [11]:
tester_model2.set_weights(best_weights)

In [13]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.806, 0.92, 0.672, 0.477, 0.694, 0.608, 0.772, 0.805, 0.852, 0.737]


In [14]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.777, 0.837, 0.626, 0.467, 0.751, 0.67, 0.812, 0.803, 0.867, 0.808]


In [10]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
oa = min(acc)
ia = np.argmin(acc)

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        K = len(set(y_test_flat))
        yp = tf.argmax(y_pred, axis=1)
        acc = []
        for i in range(K):
            a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
            acc.append(a)
        na = acc[ia]
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.7*(na - oa) + 0.3*(ol - nl) >= tol:
            best_change = 0.7*(na - oa) + 0.3*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        K = len(set(y_test_flat))
        yp = tf.argmax(y_pred, axis=1)
        acc = []
        for i in range(K):
            a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
            acc.append(a)
        na = acc[ia]
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.7*(na - oa) + 0.3*(ol - nl) >= tol:
            best_change = 0.7*(na - oa) + 0.3*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7744 - accuracy: 0.7343 - auc: 0.9650
Considering layer 5
Improvement has occured!! Accuracy: 0.574 --- Loss: 0.7743887901306152 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.574 --- Loss: 0.7743887901306152 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.013817846107482955 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Lo

In [12]:
print(f"Time to run the removal: {(end-start) / 60}")

Time to run the removal: 90.17798533837001


In [13]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.782, 0.861, 0.71, 0.589, 0.574, 0.641, 0.783, 0.728, 0.877, 0.798]


In [15]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.764, 0.702, 0.177, 0.193, 0.955, 0.619, 0.7, 0.377, 0.737, 0.862]


In [10]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
oa = min(acc)
ia = np.argmin(acc)

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        K = len(set(y_test_flat))
        yp = tf.argmax(y_pred, axis=1)
        acc = []
        for i in range(K):
            a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
            acc.append(a)
        na = acc[ia]
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.7*(na - oa) + 0.3*(ol - nl) >= tol:
            best_change = 0.7*(na - oa) + 0.3*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    current_pos = 0
    num_removed2 = 0
    nodes_removed2 = []
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        if current_pos in nodes_removed2:
            current_pos += 1
            if current_pos - num_removed2 >= size:
                print("Layer optimized")
                end_not_reached = False
            continue
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        K = len(set(y_test_flat))
        yp = tf.argmax(y_pred, axis=1)
        acc = []
        for i in range(K):
            a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
            acc.append(a)
        na = acc[ia]
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.7*(na - oa) + 0.3*(ol - nl) >= tol:
            best_change = 0.7*(na - oa) + 0.3*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            current_pos = 0
        current_pos += 1
        if current_pos - num_removed2 >= size:
            print("Layer optimized")
            end_not_reached = False
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7744 - accuracy: 0.7343 - auc: 0.9650
Considering layer 5
Improvement has occured!! Accuracy: 0.574 --- Loss: 0.7743887901306152 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.574 --- Loss: 0.7743887901306152 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.013817846107482955 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Loss: 0.9009959697723389 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.648 --- Lo

In [16]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
frac = 0.3
ignore_tol = -1e-2
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7605 - accuracy: 0.7492 - auc: 0.9653
Considering layer 5
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605359554290771 --- Change: 4.1723251342773435e-08 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 8.344650268554688e-07 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.749199

In [17]:
print(f"Time to run the removal: {(end-start) / 60}")

Time to run the removal: 110.70612378517787


In [18]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.787, 0.897, 0.654, 0.513, 0.729, 0.624, 0.842, 0.789, 0.849, 0.808]


In [19]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.783, 0.867, 0.637, 0.535, 0.727, 0.668, 0.81, 0.792, 0.871, 0.833]


In [22]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
frac = 0.3
ignore_tol = -1e-2
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:  # Ignoring very important nodes
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7605 - accuracy: 0.7492 - auc: 0.9653
Considering layer 5
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 8.761882781982421e-07 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss

In [23]:
print(f"Time to run the removal: {(end-start) / 60}")

Time to run the removal: 47.23251302242279


In [24]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.787, 0.897, 0.654, 0.513, 0.729, 0.624, 0.842, 0.789, 0.849, 0.808]


In [25]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.783, 0.867, 0.637, 0.535, 0.727, 0.668, 0.81, 0.792, 0.871, 0.833]


In [26]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
frac = 0.3
ignore_tol = -1e-3
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:  # Ignoring very important nodes
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

20/20 - 0s - loss: 0.7605 - accuracy: 0.7492 - auc: 0.9653
Considering layer 5
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 8.761882781982421e-07 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss: 0.7605347633361816 --- Change: 0.0 --- New tol: -1e-05
Improvement has occured!! Accuracy: 0.7491999864578247 --- Loss

In [27]:
print(f"Time to run the removal: {(end-start) / 60}")

Time to run the removal: 12.586614600817363


In [28]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.787, 0.897, 0.654, 0.513, 0.729, 0.624, 0.842, 0.789, 0.849, 0.808]


In [29]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.766, 0.87, 0.632, 0.547, 0.711, 0.669, 0.828, 0.795, 0.876, 0.837]


In [ ]:
fp = tf.keras.metrics.FalsePositives()
tp = tf.keras.metrics.TruePositives()
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
tol = -1e-5
ignore_tol = -1e-2
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
amounts = []
places = []
y_pred = model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class_5 = [1 if y == 4 else 0 for y in y_pred_flat]
y_test_class_5 = [1 if y == 4 else 0 for y in y_test_flat]

fp.update_state(y_test_class_5, y_pred_class_5)
tp.update_state(y_test_class_5, y_pred_class_5)

ofp = fp.result().numpy()
otp = tp.result().numpy()

fp.reset_states()
tp.reset_states()

start = time.time()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        y_pred_flat = np.argmax(y_pred, axis=1)
        y_pred_class_5 = [1 if y == 4 else 0 for y in y_pred_flat]

        fp.update_state(y_test_class_5, y_pred_class_5)
        tp.update_state(y_test_class_5, y_pred_class_5)
        
        nfp = fp.result().numpy()
        ntp = tp.result().numpy()
        fp.reset_states()
        tp.reset_states()
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.5*(ntp - otp) + 0.5*(ofp - nfp) >= tol:
            best_change = 0.5*(ntp - otp) + 0.5*(ofp - nfp)
            ol = nl
            oa = na
            otp = ntp
            ofp = nfp
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! True Positives:", ntp, "--- False Positives:", nfp, "--- Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif 0.5*(ntp - otp) + 0.5*(ofp - nfp) <= ignore_tol:  # Ignoring very important nodes
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        y_pred_flat = np.argmax(y_pred, axis=1)
        y_pred_class_5 = [1 if y == 4 else 0 for y in y_pred_flat]

        fp.update_state(y_test_class_5, y_pred_class_5)
        tp.update_state(y_test_class_5, y_pred_class_5)
        
        nfp = fp.result().numpy()
        ntp = tp.result().numpy()
        fp.reset_states()
        tp.reset_states()
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if 0.5*(ntp - otp) + 0.5*(ofp - nfp) >= tol:
            best_change = 0.5*(ntp - otp) + 0.5*(ofp - nfp)
            ol = nl
            oa = na
            otp = ntp
            ofp = nfp
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! True Positives:", ntp, "--- False Positives:", nfp, "--- Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif 0.5*(ntp - otp) + 0.5*(ofp - nfp) <= ignore_tol:
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

In [9]:
print(len(y_test_flat[y_test_flat==4]))

1000


In [10]:
y_test_class_5 = [1 if y == 4 else 0 for y in y_test_flat]

In [11]:
print(f"Time to run the removal: {(end-start) / 60}")

Time to run the removal: 6.011486442883809


In [12]:
fp = tf.keras.metrics.FalsePositives()
tp = tf.keras.metrics.TruePositives()

y_pred = tester_model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class_5 = [1 if y == 4 else 0 for y in y_pred_flat]
y_test_class_5 = [1 if y == 4 else 0 for y in y_test_flat]

fp.update_state(y_test_class_5, y_pred_class_5)
tp.update_state(y_test_class_5, y_pred_class_5)

print(fp.result().numpy())
print(tp.result().numpy())

205.0
626.0


In [39]:
fp = tf.keras.metrics.FalsePositives()
tp = tf.keras.metrics.TruePositives()

y_pred = model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class_5 = np.array([1 if y == 4 else 0 for y in y_pred_flat])
y_test_class_5 = np.array([1 if y == 4 else 0 for y in y_test_flat])

acc = (y_pred_class_5 == y_test_class_5).mean()
print(acc)
print(y_pred_class_5 == y_test_class_5)

fp.update_state(y_test_class_5, y_pred_class_5)
tp.update_state(y_test_class_5, y_pred_class_5)

print(fp.result().numpy())
print(tp.result().numpy())

0.9405
[ True  True  True ...  True False  True]
323.0
728.0


In [1]:
loss, acc, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=512)
original2 = model2.get_weights()
weight_len = len(original2) - 3
frac = 0.75
tol = -1e-5
ignore_tol = -1e-3
k = 9
dense_layer_sizes = [64]
conv_layer_sizes = [256, 128, 64, 32]
conv_len = weight_len - 2 * len(dense_layer_sizes)
bas2 = [acc]
bls2 = [loss]
best_weights = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
num_removed2 = 0
amount = []
place = []
y_pred = model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class = np.array([1 if y == k else 0 for y in y_pred_flat])
y_test_class = np.array([1 if y == k else 0 for y in y_test_flat])

oa = (y_pred_class == y_test_class).mean()
for layer, size in enumerate(dense_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(dense_layer_sizes+conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[weight_len - (2*layer+1)][:,current_pos] = 0
        w[weight_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        y_pred_flat = np.argmax(y_pred, axis=1)
        y_pred_class = np.array([1 if y == k else 0 for y in y_pred_flat])

        na = (y_pred_class == y_test_class).mean()
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            dense_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[weight_len - (2*layer+1)][:,current_pos] = 0
            best_weights[weight_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:  # Ignoring very important nodes
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amount.append(num_removed2)
    place.append(nodes_removed2)


for layer, size in enumerate(conv_layer_sizes):
    end_not_reached = True
    num_removed2 = 0
    nodes_removed2 = []
    nodes_included = list(np.arange(size))
    current_pos = nodes_included[0]
    idx = 0
    print(f'Considering layer {len(conv_layer_sizes) - layer}')
    while end_not_reached:
        w = copy.deepcopy(best_weights)
        w[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
        w[conv_len - 2*layer][current_pos] = 0
        tester_model2.set_weights(w)
        del w
        nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
        y_pred = tester_model2.predict(x_test)
        y_pred_flat = np.argmax(y_pred, axis=1)
        y_pred_class = np.array([1 if y == k else 0 for y in y_pred_flat])

        na = (y_pred_class == y_test_class).mean()
        # print(f"Node {current_pos}:", 0.*(na - oa) + 1.*(ol - nl))
        if frac*(na - oa) + (1.-frac)*(ol - nl) >= tol:
            best_change = frac*(na - oa) + (1.-frac)*(ol - nl)
            ol = nl
            oa = na
            size -= 1
            conv_layer_sizes[layer] -= 1
            nodes_removed2 += [current_pos]
            nodes_included.remove(current_pos)
            best_weights[conv_len - (2*layer+1)][:,:,:,current_pos] = 0
            best_weights[conv_len - 2*layer][current_pos] = 0
            num_removed2 += 1
            print("Improvement has occured!! Accuracy:", na, "--- Loss:", nl, '--- Change:', best_change, '--- New tol:', tol)
            idx = 0
        elif frac*(na - oa) + (1.-frac)*(ol - nl) <= ignore_tol:
            size -= 1
            nodes_included.remove(current_pos)
            idx += 1
        else:
            idx += 1
        if idx >= size:
            print("Layer optimized")
            end_not_reached = False
        else:
            current_pos = nodes_included[idx]
    amounts.append(num_removed2)
    places.append(nodes_removed2)

end = time.time()
tester_model2.set_weights(best_weights)

NameError: name 'model2' is not defined

In [66]:
fp = tf.keras.metrics.FalsePositives()
tp = tf.keras.metrics.TruePositives()

y_pred = tester_model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class_5 = [1 if y == 4 else 0 for y in y_pred_flat]
y_test_class_5 = [1 if y == 4 else 0 for y in y_test_flat]

fp.update_state(y_test_class_5, y_pred_class_5)
tp.update_state(y_test_class_5, y_pred_class_5)

print(fp.result().numpy())
print(tp.result().numpy())

388.0
731.0


In [67]:
fp = tf.keras.metrics.FalsePositives()
tp = tf.keras.metrics.TruePositives()

y_pred = model2.predict(x_test)
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_class_5 = np.array([1 if y == 4 else 0 for y in y_pred_flat])
y_test_class_5 = np.array([1 if y == 4 else 0 for y in y_test_flat])


fp.update_state(y_test_class_5, y_pred_class_5)
tp.update_state(y_test_class_5, y_pred_class_5)

print(fp.result().numpy())
print(tp.result().numpy())

442.0
774.0


In [68]:
y_pred = model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.716, 0.765, 0.574, 0.653, 0.774, 0.559, 0.821, 0.775, 0.888, 0.838]


In [69]:
y_pred = tester_model2.predict(x_test)
K = len(set(y_test_flat))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test_flat == i] == y_test_flat[y_test_flat == i]).numpy())
    acc.append(a)
accuracies = tf.convert_to_tensor(acc)
print(acc)

[0.773, 0.856, 0.599, 0.577, 0.731, 0.608, 0.816, 0.79, 0.848, 0.812]


# Class Specific removals

In [13]:
class1 = [amounts, places]

In [19]:
class2 = [amounts, places]

In [25]:
class3 = [amounts, places]

In [34]:
class4 = [amounts, places]

In [40]:
class5 = [amounts, places]

In [46]:
class6 = [amounts, places]

In [52]:
class7 = [amounts, places]

In [58]:
class8 = [amounts, places]

In [64]:
class9 = [amounts, places]

In [70]:
class10 = [amounts, places]

In [26]:
class1

[[38, 116, 22, 7, 3],
 [[0,
   2,
   3,
   4,
   6,
   7,
   9,
   10,
   11,
   12,
   17,
   20,
   21,
   22,
   23,
   25,
   26,
   28,
   29,
   30,
   33,
   34,
   35,
   37,
   39,
   40,
   42,
   45,
   47,
   49,
   50,
   51,
   52,
   56,
   58,
   59,
   60,
   61],
  [0,
   4,
   5,
   2,
   6,
   7,
   8,
   10,
   13,
   12,
   15,
   16,
   17,
   18,
   23,
   26,
   27,
   28,
   19,
   21,
   9,
   30,
   37,
   40,
   45,
   48,
   25,
   39,
   24,
   31,
   43,
   32,
   51,
   53,
   60,
   63,
   64,
   65,
   69,
   70,
   52,
   58,
   57,
   72,
   74,
   76,
   79,
   80,
   86,
   92,
   99,
   102,
   110,
   114,
   116,
   118,
   119,
   124,
   125,
   126,
   128,
   129,
   134,
   138,
   144,
   145,
   153,
   156,
   83,
   123,
   94,
   78,
   159,
   162,
   163,
   165,
   167,
   169,
   171,
   182,
   183,
   181,
   184,
   185,
   187,
   188,
   192,
   194,
   195,
   203,
   205,
   210,
   211,
   215,
   221,
   222,
   223,
   2

In [27]:
class2

[[38, 111, 22, 7, 1],
 [[0,
   3,
   2,
   4,
   7,
   6,
   10,
   9,
   11,
   12,
   17,
   20,
   21,
   23,
   22,
   26,
   25,
   29,
   28,
   30,
   33,
   35,
   34,
   40,
   37,
   39,
   45,
   42,
   50,
   47,
   49,
   51,
   52,
   59,
   58,
   56,
   60,
   61],
  [0,
   4,
   6,
   7,
   8,
   2,
   10,
   13,
   1,
   9,
   16,
   17,
   21,
   23,
   18,
   26,
   27,
   31,
   32,
   37,
   40,
   48,
   36,
   45,
   53,
   12,
   60,
   63,
   64,
   51,
   5,
   30,
   58,
   65,
   69,
   72,
   74,
   76,
   78,
   79,
   80,
   86,
   87,
   92,
   102,
   99,
   110,
   114,
   116,
   118,
   119,
   38,
   15,
   39,
   19,
   124,
   125,
   126,
   128,
   129,
   144,
   134,
   138,
   24,
   145,
   153,
   159,
   162,
   163,
   167,
   165,
   169,
   182,
   184,
   171,
   185,
   188,
   187,
   194,
   192,
   195,
   205,
   210,
   211,
   221,
   215,
   222,
   223,
   226,
   225,
   229,
   94,
   200,
   85,
   14,
   66,
   105,
   16

In [28]:
class3

[[39, 116, 24, 8, 1],
 [[0,
   3,
   2,
   4,
   7,
   6,
   10,
   9,
   11,
   12,
   17,
   20,
   21,
   23,
   22,
   26,
   25,
   29,
   28,
   30,
   33,
   35,
   34,
   40,
   37,
   39,
   44,
   42,
   45,
   50,
   47,
   49,
   51,
   52,
   59,
   58,
   56,
   60,
   61],
  [0,
   1,
   3,
   4,
   6,
   7,
   8,
   5,
   10,
   16,
   12,
   13,
   17,
   18,
   25,
   26,
   27,
   32,
   21,
   23,
   9,
   19,
   28,
   30,
   31,
   22,
   37,
   36,
   15,
   38,
   39,
   40,
   43,
   45,
   48,
   24,
   53,
   54,
   60,
   63,
   65,
   72,
   69,
   74,
   76,
   79,
   80,
   92,
   86,
   102,
   99,
   110,
   114,
   118,
   116,
   124,
   125,
   126,
   128,
   129,
   138,
   134,
   145,
   153,
   144,
   159,
   162,
   163,
   165,
   180,
   169,
   171,
   182,
   34,
   78,
   123,
   75,
   58,
   103,
   183,
   184,
   185,
   188,
   187,
   195,
   192,
   194,
   200,
   2,
   66,
   59,
   94,
   164,
   64,
   205,
   210,
   211,
   2

In [ ]:
tf.save_weights()